# Transforming Housing Units from Municipality to Grid

In [1]:
%load_ext jupyter_black
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [66]:
base_url = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/"
)
impact_input_dir = base_url / "input/"
input_dir = base_url / "input/Google Footprint Data/"
output_dir = base_url / "output/"

In [53]:
phl_ggl_bld_municip_count = pd.read_csv(
    input_dir / "phl_google_bld_municip_count.csv"
)
phl_ggl_bld_grid_count = pd.read_csv(
    input_dir / "phl_google_bld_grid_count.csv"
)
phl_ggl_bld_intersection_count = pd.read_csv(
    input_dir / "phl_google_bld_intersection_count.csv"
)

## Computing weights

In [54]:
## adding a building to the municipality pcode = PH175321000 and grid id = 101 and Centroid  = 114.3E_11.1N
## there are no buildings in this municipality and is an island in the ocean
# This is done to compute the weights and ensure housing units are not left out.
# It is the only municipality in the grid.
# phl_ggl_bld_municip_count.loc[len(phl_ggl_bld_municip_count.index)] = ['Amy', 89, 93]
phl_ggl_bld_municip_count[
    phl_ggl_bld_municip_count["ADM3_PCODE"] == "PH175321000"
]
phl_ggl_bld_grid_count[phl_ggl_bld_grid_count["id"] == 101]
phl_ggl_bld_intersection_count[
    (phl_ggl_bld_intersection_count["ADM3_PCODE"] == "PH175321000")
    & (phl_ggl_bld_intersection_count["id"] == 101)
]

,ADM3_PCODE,id,Centroid,numbuildings
9766,PH175321000,101.0,114.3E_11.1N,0


In [55]:
phl_ggl_bld_municip_count.loc[
    phl_ggl_bld_municip_count["ADM3_PCODE"] == "PH175321000", "numbuildings"
] = 1
phl_ggl_bld_grid_count.loc[
    phl_ggl_bld_grid_count["id"] == 101, "numbuildings"
] = 1
phl_ggl_bld_intersection_count.loc[
    (phl_ggl_bld_intersection_count["ADM3_PCODE"] == "PH175321000")
    & (phl_ggl_bld_intersection_count["id"] == 101),
    "numbuildings",
] = 1

### Municipality to Grid

In [56]:
mun_to_grid = phl_ggl_bld_intersection_count.merge(
    phl_ggl_bld_municip_count, on="ADM3_PCODE", suffixes=("_x", None)
)
mun_to_grid["weight"] = (
    mun_to_grid["numbuildings_x"] / mun_to_grid["numbuildings"]
)
mun_to_grid[mun_to_grid["Centroid"] == "122.1E_17.4N"]

,ADM3_PCODE,id,Centroid,numbuildings_x,numbuildings,weight
850,PH023106000,13064.0,122.1E_17.4N,0,26094,0.000000
956,PH023117000,13064.0,122.1E_17.4N,1,1578,0.000634
1069,PH023133000,13064.0,122.1E_17.4N,0,13510,0.000000
1091,PH023137000,13064.0,122.1E_17.4N,0,35325,0.000000


In [57]:
mun_to_grid.to_csv(input_dir / "ggl_mun_to_grid_weights.csv", index=False)

### Grid to Municipality

In [58]:
grid_to_mun = phl_ggl_bld_intersection_count.merge(
    phl_ggl_bld_grid_count, on="Centroid", suffixes=("_x", None)
)
grid_to_mun["weight"] = (
    grid_to_mun["numbuildings_x"] / grid_to_mun["numbuildings"]
)
grid_to_mun.groupby("Centroid").sum().sort_values(by="weight")

C:\Users\pauni\AppData\Local\Temp\ipykernel_5692\1780964883.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grid_to_mun.groupby("Centroid").sum().sort_values(by="weight")


,id_x,numbuildings_x,id,numbuildings,weight
Centroid,,,,,
119.2E_11.0N,8285.0,0,8285,0,0.0
122.0E_13.6N,12935.0,0,12935,0,0.0
123.9E_11.4N,16130.0,0,16130,0,0.0
123.9E_12.3N,16121.0,0,16121,0,0.0
122.3E_18.5N,13387.0,0,13387,0,0.0
...,...,...,...,...,...
121.6E_18.0N,48892.0,9377,48892,37508,1.0
121.6E_18.1N,48888.0,12910,48888,51640,1.0
121.6E_18.2N,36663.0,9726,36663,29178,1.0


In [59]:
grid_to_mun.to_csv(input_dir / "ggl_grid_to_mun_weights.csv", index=False)

## Transforming the Housing Units data

The housing units data is from `https://data.humdata.org/dataset/philippines-pre-disaster-indicators`

- File URL `https://data.humdata.org/dataset/f26a0a04-0549-4139-af91-81dfa6e56082/resource/557b601f-e2f5-42ef-8742-e47395427384/download/180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx`

In [60]:
construction_materials_df = pd.read_excel(
    base_url
    / "input/180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx",
    sheet_name="by category",
)
households_df = pd.read_excel(
    base_url / "input/180814_number-of-household_by-city_municipality.xlsx",
    sheet_name="Data",
)

In [61]:
# filling in missing housing units with households
housing_units_df = construction_materials_df[
    ["Municipality_City Code", "Housing Units"]
].merge(households_df, on="Municipality_City Code", how="left")
# using right join to preserve all municipalities
hu_grid = housing_units_df.merge(
    mun_to_grid[["ADM3_PCODE", "id", "Centroid", "weight"]],
    how="right",
    left_on="Municipality_City Code",
    right_on="ADM3_PCODE",
)
hu_grid[hu_grid["Housing Units"].isna()]

,Municipality_City Code,Housing Units,Region,Region Code,Province,Province Code,Municipality_City,Number of Household,ADM3_PCODE,id,Centroid,weight
7389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133901000,11088.0,120.9E_14.6N,0.012593
7390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133901000,11255.0,121.0E_14.6N,0.987407
7391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133902000,11255.0,121.0E_14.6N,1.000000
7392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133903000,11255.0,121.0E_14.6N,1.000000
7393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133904000,11255.0,121.0E_14.6N,1.000000
7394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133905000,11255.0,121.0E_14.6N,1.000000
7395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133906000,11255.0,121.0E_14.6N,1.000000
7396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133907000,11255.0,121.0E_14.6N,1.000000
7397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133908000,11255.0,121.0E_14.6N,1.000000
7398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH133909000,11255.0,121.0E_14.6N,1.000000


In [62]:
hu_grid["Housing Units"].fillna(hu_grid["Number of Household"], inplace=True)
hu_grid["Housing Units"].fillna(1, inplace=True)
hu_grid[hu_grid["Housing Units"].isna()]
# should not show rows

,Municipality_City Code,Housing Units,Region,Region Code,Province,Province Code,Municipality_City,Number of Household,ADM3_PCODE,id,Centroid,weight


In [63]:
# multiplying by weights
hu_grid["hu_bygrid"] = hu_grid["Housing Units"] * hu_grid["weight"]
hu_grid_df = hu_grid.groupby(["id", "Centroid"]).sum().reset_index()
hu_grid_df.drop(["Housing Units", "weight"], axis=1, inplace=True)
hu_grid_df["hu_bygrid"].sum()

C:\Users\pauni\AppData\Local\Temp\ipykernel_5692\635578274.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hu_grid_df = hu_grid.groupby(["id", "Centroid"]).sum().reset_index()


22011220.0

In [64]:
transformed_df = phl_ggl_bld_grid_count.merge(
    hu_grid_df, on=["id", "Centroid"]
)
transformed_df.to_csv(
    output_dir / "transformed_housingunits_bygrid.csv", index=False
)

In [79]:
# Percentage Damage by Grid Using these Weights
build_dmg_data = pd.read_csv(
    impact_input_dir / "IMpact_data_philipines_SEP_2021.csv"
)
build_dmg_data.drop("Id", axis=1, inplace=True)
build_dmg_data.drop_duplicates(
    subset=["pcode", "typhoon", "Year"], inplace=True
)

In [87]:
# Not all municipalities are in the damage data set.
# Not all municipalities in the building damage data can be found in the admin 3 shapefile.
# removing those buildings with incorrect pcode in them
build_dmg_data_grouped = build_dmg_data[
    build_dmg_data["pcode"].isin(list(phl_ggl_bld_municip_count["ADM3_PCODE"]))
]
build_dmg_data_grouped["Totally"].sum()

1670866.0

In [88]:
build_dmg_data_grouped = build_dmg_data_grouped.merge(
    mun_to_grid,
    left_on="pcode",
    right_on="ADM3_PCODE",
    how="right",
    suffixes=("_x", None),
)
# multiplying by weights
build_dmg_data_grouped["damaged_bygrid"] = (
    build_dmg_data_grouped["Totally"] * build_dmg_data_grouped["weight"]
)
build_dmg_data_grouped

,pcode,typhoon,Year,Totally,Partially,total,ADM3_PCODE,id,Centroid,numbuildings_x,numbuildings,weight,damaged_bygrid
0,PH012801000,Fung-wong,2014.0,0.0,5.0,5.0,PH012801000,11049.0,120.9E_18.5N,1052,1245,0.844980,0.000000
1,PH012801000,Goni,2015.0,0.0,30.0,30.0,PH012801000,11049.0,120.9E_18.5N,1052,1245,0.844980,0.000000
2,PH012801000,Sarika,2016.0,1.0,30.0,31.0,PH012801000,11049.0,120.9E_18.5N,1052,1245,0.844980,0.844980
3,PH012801000,Haima,2016.0,1.0,50.0,51.0,PH012801000,11049.0,120.9E_18.5N,1052,1245,0.844980,0.844980
4,PH012801000,Mangkhut,2018.0,1.0,297.0,298.0,PH012801000,11049.0,120.9E_18.5N,1052,1245,0.844980,0.844980
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29551,PH175917000,Rammasun,2014.0,1.0,12.0,13.0,PH175917000,13114.0,122.1E_12.4N,1820,1892,0.961945,0.961945
29552,PH175917000,Melor,2015.0,0.0,5.0,5.0,PH175917000,13114.0,122.1E_12.4N,1820,1892,0.961945,0.000000
29553,PH175917000,Haiyan,2013.0,7.0,42.0,49.0,PH175917000,13114.0,122.1E_12.4N,1820,1892,0.961945,6.733615
29554,PH175917000,PHANFONE,2019.0,8.0,26.0,34.0,PH175917000,13114.0,122.1E_12.4N,1820,1892,0.961945,7.695560


In [89]:
build_dmg_data_grouped = build_dmg_data_grouped.groupby(
    ["id", "Centroid", "typhoon", "Year"], as_index=False
).sum()
build_dmg_data_grouped

C:\Users\pauni\AppData\Local\Temp\ipykernel_5692\2321245874.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  build_dmg_data_grouped = build_dmg_data_grouped.groupby(


,id,Centroid,typhoon,Year,Totally,Partially,total,numbuildings_x,numbuildings,weight,damaged_bygrid
0,6133.0,117.9E_9.1N,LINFA,2015.0,0.0,1.0,1.0,0,17330,0.000000,0.000000
1,6134.0,117.9E_9.0N,LINFA,2015.0,0.0,1.0,1.0,20,17330,0.001154,0.000000
2,6299.0,118.0E_9.2N,LINFA,2015.0,0.0,1.0,1.0,328,17330,0.018927,0.000000
3,6300.0,118.0E_9.1N,LINFA,2015.0,0.0,1.0,1.0,2332,17330,0.134564,0.000000
4,6301.0,118.0E_9.0N,LINFA,2015.0,0.0,1.0,1.0,6190,17330,0.357184,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
14123,20680.0,126.6E_7.3N,Haiyan,2013.0,3.0,0.0,3.0,90,11950,0.007531,0.022594
14124,20680.0,126.6E_7.3N,Lingling,2014.0,62.0,0.0,62.0,2798,26194,0.197647,11.239387
14125,20681.0,126.6E_7.2N,Bopha,2012.0,80.0,1500.0,1580.0,468,11950,0.039163,3.133054
14126,20681.0,126.6E_7.2N,Haiyan,2013.0,3.0,0.0,3.0,468,11950,0.039163,0.117490


In [90]:
build_dmg_data_grouped["damaged_bygrid"].sum()

1670866.0

In [92]:
build_dmg_data_grouped.to_csv(
    output_dir / "building_damage_bygrid_gglfpdata.csv", index=False
)